# Locating a fusion restaurant in Los Angeles

In [9]:
import pandas as pd
import numpy as np
import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library

VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
CLIENT_ID='HM3AAQI50VK0VMIFVRGE1WATLARDR3EHLEYY1RAVJFN3PRKE'
CLIENT_SECRET='ENWI1Y4UVIVQ13MEKRL5FKYYTHNE5JAXRF2JASLIOG05JRX4'
ACCESS_TOKEN='2I3X5M1HPK3OJEM14M3VXUOEKFQ0VSY2BUYIC2N0D5L4THGZ'

In [4]:
from geopy.geocoders import Nominatim
address = 'Los Angeles, CA'

geolocator = Nominatim(user_agent="la_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Los Angeles are {}, {}.'.format(latitude, longitude)) 

The geograpical coordinate of Los Angeles are 34.0536909, -118.242766.


In [6]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


In [10]:
search_query = 'Restaurant'
radius = 200000
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()
venues = results['response']['venues']
df = json_normalize(venues)
filtered_columns = ['name', 'categories'] + [col for col in df.columns if col.startswith('location.')] + ['id']
df_filtered = df.loc[:, filtered_columns]
# filter the category for each row
df_filtered['categories'] = df_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
df_filtered.columns = [column.split('.')[-1] for column in df_filtered.columns]

def unique_list(input_list):
    output_list = []
    for word in input_list:
        if word not in output_list:
            output_list.append(word)
    print(output_list)
    return output_list
Category_list=unique_list(df_filtered.categories)

['French Restaurant', 'American Restaurant', 'New American Restaurant', 'Mexican Restaurant', 'Chinese Restaurant', 'Seafood Restaurant', 'Italian Restaurant', 'Salad Place', 'Asian Restaurant', 'Latin American Restaurant', 'Kitchen Supply Store', 'Restaurant', 'Sushi Restaurant', 'Mediterranean Restaurant', 'Indian Restaurant', 'Japanese Restaurant', 'Spanish Restaurant', 'Construction & Landscaping', 'Food', None, 'Thai Restaurant', 'Taco Place', 'Korean Restaurant', 'Breakfast Spot', 'Diner', 'Cajun / Creole Restaurant']


In [16]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around Ecco


# add Ecco as a red circle mark
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)

# add popular spots to the map as blue circle markers
for lat, lng, label in zip(df_filtered.lat, df_filtered.lng, df_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='skyblue',
        fill_color='skyblue',
        fill_opacity=0.6
        ).add_to(venues_map)
    
for lat, lng, label in zip(df_filtered.lat, df_filtered.lng, df_filtered.categories):
    if label=='Chinese Restaurant': 
        
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
            ).add_to(venues_map)
    
for lat, lng, label in zip(df_filtered.lat, df_filtered.lng, df_filtered.categories):
    if label=='Korean Restaurant': 
        
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            fill=True,
            color='green',
            fill_color='green',
            fill_opacity=0.6
            ).add_to(venues_map)
# display map
venues_map